## Recommending movies using Collaborative Filtering 

In [74]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import hamming

In [75]:
import warnings
warnings.simplefilter(action='ignore',category=Warning)

In [76]:
df_ratings = pd.read_csv("recent_ratings.csv")
df_movies = pd.read_csv('recent_movies.csv')

In [77]:
df_ratings.shape, df_movies.shape

((552, 4), (188, 4))

In [78]:
df_ratings.userId.unique().size  # No. of unique users 

63

In [79]:
df_ratings.movieId.unique().size  # No. of unique movies 

188

In [80]:
df_ratings.sample(5)

,userId,movieId,rating,timestamp
478,567,183897,3.5,1525282061
162,184,176371,4.5,1537109683
121,111,184015,3.5,1518640587
181,205,179817,4.5,1519901378
133,153,168492,4.5,1525548280


In [81]:
# Remove timestamp column  
df_ratings.drop(columns='timestamp', inplace=True)

In [82]:
df_ratings.sample(5)

,userId,movieId,rating
388,414,180985,3.5
244,249,170875,3.0
121,111,184015,3.5
6,18,180095,3.0
261,258,122916,3.0


In [83]:
df_movies.sample(5)

,movieId,title,genres,year
168,187717,Won't You Be My Neighbor? (2018),Documentary,2018
151,184641,Fullmetal Alchemist 2018 (2017),Action|Adventure|Fantasy,2017
133,181659,Craig Ferguson: Tickle Fight (2017),Comedy,2017
121,180045,Molly's Game (2017),Drama,2017
64,173291,Valerian and the City of a Thousand Planets (2...,Action|Adventure|Sci-Fi,2017


In [84]:
# Get rating for each user and movie - userid is row label and movieid is column label 
ratings = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating')

In [85]:
ratings.shape

(63, 188)

In [86]:
ratings.sample(5)

movieId,122896,122898,122906,122912,122916,122918,122926,143355,166534,167064,...,189381,189713,190183,190209,190215,191005,193581,193583,193585,193587
userId,,,,,,,,,,,,,,,,,,,,,
305,NaN,NaN,NaN,4.5,4.5,3.5,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,NaN,NaN,4.0,NaN,4.0,4.0,4.5,4.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
ratings.loc[233, ratings.loc[233,:].notna()]  # Ratings given by 233 

movieId
122912    2.0
168266    3.5
174055    2.5
177593    5.0
178061    3.0
180031    3.5
183011    1.5
183897    3.5
187593    2.5
Name: 233, dtype: float64

### Hamming Distance
Measures how different two sequences are. It is <b>% of disagreement </b>. A value of 1 indicates sequences are very different, 0 indicates they are very similar.

In [88]:
l1 = (1,2,4,np.nan)
l2 = (1,2,np.nan,3)
l3 = (1,np.nan, 3,5)
l4 = (np.nan, np.nan, np.nan, np.nan)
print(hamming(l1,l2))
print(hamming(l1,l3))
print(hamming(l1,l4))

0.5
0.75
1.0


In [89]:
# Find out hamming distance between ratings of two users
def hamming_distance(ratings, user1, user2):
    # Ratings of a user 
    try:
        user1_ratings = ratings.loc[user1,:]
        user2_ratings = ratings.loc[user2,:]
        distance = hamming(user1_ratings, user2_ratings)
    except:
        distance = np.NaN

    return distance    

In [90]:
# Get k neighbours of the given user based on hamming distance 
def get_nearest_users(ratings, active_user, k = 10):
    all_users = pd.DataFrame(ratings.index) # UserIds 
    other_users = all_users[all_users.userId != active_user]
    
    other_users['distance'] = other_users['userId'].apply(lambda userid: hamming_distance(ratings,active_user,userid))
    
    # find out hamming distance and return users with low hamming distance from active user 
    return  other_users.sort_values(['distance'], ascending = True).userId[:k]

In [91]:
print (get_nearest_users(ratings, 184))

22    249
28    305
19    212
2      25
57    567
30    318
8      98
49    514
50    515
0      18
Name: userId, dtype: int64


In [92]:
def get_recommended_movies(ratings, movies, user, top=5):
    # Find out nearest neighbours based on hamming distance 
    nn_users = get_nearest_users(ratings, user, 10)
    #print(nn_users)
    # Get ratings of nearest neighbours(users)
    nn_ratings = ratings[ratings.index.isin(nn_users)]
    
    # Average ratings given by nearest neighbours for all movies
    avg_ratings = nn_ratings.apply(np.nanmean).dropna()
    #print(avg_ratings.shape)
    #print(avg_ratings.sort_values(ascending=False)[:20])
    
    # Find out movies that user had already watched
    movies_watched = ratings.transpose()[user].dropna().index
    
    # remove movies that user already watched
    avg_ratings = avg_ratings[~ avg_ratings.index.isin(movies_watched)]
    
    # Findout top n movies based on avg ratings given by nearest neighbours 
    top_movies_ids = avg_ratings.sort_values(ascending=False).index[:top]
    print(top_movies_ids)
    # Return recommended movies 
    return movies[movies.movieId.isin(top_movies_ids)].title

In [93]:
get_recommended_movies(ratings, df_movies, 249, 5)

Index([172705, 178827, 183897, 177593, 168418], dtype='int64', name='movieId')


22                                 The Boss Baby (2017)
56                               Tickling Giants (2017)
97     Three Billboards Outside Ebbing, Missouri (2017)
105                                 Paddington 2 (2017)
145                                 Isle of Dogs (2018)
Name: title, dtype: object

In [94]:
get_recommended_movies(ratings, df_movies, 433,10)

Index([168326, 122912, 167746, 168248, 168492, 176371, 171763, 176751, 177593,
       173197],
      dtype='int64', name='movieId')


3               Avengers: Infinity War - Part I (2018)
12                        The Lego Batman Movie (2017)
15                       John Wick: Chapter Two (2017)
20                                 The Big Sick (2017)
23                         Call Me by Your Name (2017)
51                                  Baby Driver (2017)
58                                   The Square (2017)
87                            Blade Runner 2049 (2017)
92                                American Made (2017)
97    Three Billboards Outside Ebbing, Missouri (2017)
Name: title, dtype: object

In [95]:
get_recommended_movies(ratings, df_movies, 125, 10)

Index([177593, 185031, 166534, 173197, 187593, 122912, 185029, 173925, 122926,
       122898],
      dtype='int64', name='movieId')


1                                 Justice League (2017)
3                Avengers: Infinity War - Part I (2018)
6                     Untitled Spider-Man Reboot (2017)
8                                          Split (2017)
58                                    The Square (2017)
65                                 Seven Sisters (2017)
97     Three Billboards Outside Ebbing, Missouri (2017)
157                                A Quiet Place (2018)
158                                        Alpha (2018)
166                                   Deadpool 2 (2018)
Name: title, dtype: object